In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
ASTRA_DB_ID = os.getenv("ASTRA_DB_ID")
ASTRA_DB_TOKEN = os.getenv("ASTRA_DB_TOKEN")
OPENAI_KEY = os.getenv("OPENAI_KEY")

In [ ]:
import cassio
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from datasets import load_dataset
from PyPDF2 import PdfReader


In [ ]:
pdfReader = PdfReader('budget_speech.pdf')

In [ ]:
# reading text from the pdf

text = ''
for index, page in enumerate(pdfReader.pages):
    content = page.extract_text()
    if content:
        text += content

text